In [1]:
import sys
import os

SCRIPT_DIR = os.path.dirname(os.path.abspath('src'))
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [2]:
# Load packages
import pandas as pd
import numpy as np

import sklearn.metrics as skm
import sklearn.preprocessing as skp
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from src.utils import MlflowUtils
from sklearn.pipeline import make_pipeline
from modelingUtlis import ModelingUtils
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff


In [3]:
LOAD_PATH = '../data/interim/'
LOAD_SKILLS_DEV = '6.0-Engineered_data-split_roles-cluster_skills.pkl'
SAVE_DF_NAME = '7.0-Chosen_features_and_roles.pkl'


# Load Data

In [4]:
skills_dev_df = pd.read_pickle(LOAD_PATH + LOAD_SKILLS_DEV)

In [5]:
skills_dev_df

Languages                                                             \
            APL Assembly Bash/Shell    C    C#   C++ COBOL Clojure Crystal   
2           0.0      0.0        0.0  0.0  0.75  0.75   0.0     0.0     0.0   
3           0.0      0.0        0.0  0.0  1.50  0.00   0.0     0.0     0.0   
7           0.5      0.0        0.5  1.0  0.00  1.00   0.0     0.0     0.0   
9           0.0      0.0        0.0  0.0  0.00  0.00   0.0     0.0     0.0   
10          0.0      0.0        1.0  0.0  1.50  0.00   0.0     0.0     0.0   
...         ...      ...        ...  ...   ...   ...   ...     ...     ...   
73262       0.0      0.5        0.0  0.5  0.00  0.75   0.0     0.0     0.0   
73263       0.0      0.0        1.5  0.0  0.00  0.00   0.0     0.0     0.0   
73264       0.0      0.0        1.0  0.0  0.00  0.00   0.0     0.0     0.0   
73265       0.0      0.0        0.0  0.0  0.25  0.00   0.0     0.0     0.0   
73266       0.0      0.0        0.0  0.0  1.00  0.00   0.0     0.0     0.0   

            ...         DevType                                        \
      Dart  ... full_stack_Java full_stack_.JavaScript full_stack_PHP   
2      0.0  ...               0                      0              0   
3      0.0  ...               0                      0              0   
7      0.0  ...               1                      1              1   
9      0.0  ...               0                      0              0   
10     0.0  ...               0                      0              0   
...    ...  ...             ...                    ...            ...   
73262  0.0  ...               0                      0              0   
73263  1.0  ...               0                      0              0   
73264  0.0  ...               0                      0              0   
73265  0.0  ...               0                      0              1   
73266  0.0  ...               0                      0              0   

                                                                          \
      full_stack_python back_end_Java back_end_.JavaScript back_end_.NET   
2                     0             0                    0             0   
3                     0             0                    0             0   
7                     0             0                    0             0   
9                     0             0                    0             0   
10                    0             0                    0             1   
...                 ...           ...                  ...           ...   
73262                 0             0                    0             0   
73263                 0             0                    1             0   
73264                 0             0                    0             0   
73265                 1             0                    0             0   
73266                 0             0                    0             0   

                                                 
      back_end_C++ back_end_python back_end_PHP  
2                0               0            0  
3                0               0            0  
7                0               0            0  
9                0               0            0  
10               0               0            0  
...            ...             ...          ...  
73262            0               0            0  
73263            1               1            1  
73264            0               0            0  
73265            0               0            0  
73266            0               0            0  

[50423 rows x 228 columns]

# 1. Choosing features
we need to choose between original features or clusters features, which one gives us the best results in classification to proceed with them in hero model

we will use class weights formula:
# $$ w_j=\frac{n\_samples}{(n\_classes * n\_samples_j)} $$

In [6]:
basic_model = make_pipeline(skp.StandardScaler(), MultiOutputClassifier(LogisticRegression()))


In [7]:
original_df = skills_dev_df.drop('clustered_skills', level = 0, axis = 1)
original_modeling_utils = ModelingUtils(original_df, basic_model)
original_features_model, classification_report_original_features = original_modeling_utils.train_evaluate_model_features()

train_f1-score:  0.3983875378374043
test_f1-score:  0.3846718699548519


In [8]:
drop_columns = original_df.drop('DevType', level=0, axis=1).columns
clustered_df = skills_dev_df.drop(drop_columns, axis = 1)
clustered_modeling_utils = ModelingUtils(clustered_df, basic_model)
clustered_features_model, classification_report_clustered_features = clustered_modeling_utils.train_evaluate_model_features()

train_f1-score:  0.3205782284863643
test_f1-score:  0.3220733461768458


In [9]:
classification_report_original_features.loc['Mean',:] = classification_report_original_features.mean()
classification_report_original_features

train                      \
                                              precision    recall  f1_score   
Academic researcher                            0.462963  0.220426  0.298656   
Blockchain                                     0.440729  0.272556  0.336818   
Cloud infrastructure engineer                  0.431099  0.217780  0.289375   
Data or business analyst                       0.405914  0.124485  0.190536   
Data scientist or machine learning specialist  0.638930  0.485601  0.551813   
Database administrator                         0.336134  0.035874  0.064830   
DevOps specialist                              0.504124  0.193357  0.279508   
Developer_QA or test                           0.000000  0.000000  0.000000   
Developer_back-end                             0.596681  0.621157  0.608673   
Developer_desktop or enterprise applications   0.591538  0.166781  0.260200   
Developer_embedded applications or devices     0.592643  0.265406  0.366625   
Developer_front-end                            0.617082  0.387913  0.476369   
Developer_full-stack                           0.674350  0.731881  0.701939   
Developer_game or graphics                     0.471136  0.338688  0.394081   
Developer_mobile                               0.740981  0.570714  0.644797   
Engineer_data                                  0.519169  0.202998  0.291872   
Engineer_site reliability                      0.355212  0.141757  0.202643   
Scientist                                      0.416667  0.262195  0.321856   
Security professional                          0.213333  0.026711  0.047478   
Senior Executive (C-Suite_VP_etc.)             0.222222  0.006202  0.012066   
System administrator                           0.296875  0.052559  0.089307   
full_stack_.NET                                0.696774  0.624277  0.658537   
full_stack_Java                                0.632667  0.556927  0.592386   
full_stack_.JavaScript                         0.686091  0.704440  0.695144   
full_stack_PHP                                 0.687873  0.643125  0.664747   
full_stack_python                              0.642676  0.548254  0.591722   
back_end_Java                                  0.606329  0.617169  0.611701   
back_end_.JavaScript                           0.457177  0.467685  0.462371   
back_end_.NET                                  0.524378  0.559851  0.541534   
back_end_C++                                   0.548303  0.438321  0.487182   
back_end_python                                0.560851  0.453049  0.501219   
back_end_PHP                                   0.534188  0.492350  0.512416   
Mean                                           0.503284  0.357203  0.398388   

                                                             test            \
                                               accuracy precision    recall   
Academic researcher                            0.965070  0.398810  0.202417   
Blockchain                                     0.985845  0.363636  0.223602   
Cloud infrastructure engineer                  0.943626  0.353571  0.200405   
Data or business analyst                       0.968194  0.370370  0.131579   
Data scientist or machine learning specialist  0.965368  0.656863  0.457859   
Database administrator                         0.971392  0.241379  0.024138   
DevOps specialist                              0.937503  0.472103  0.179445   
Developer_QA or test                           0.974044  0.000000  0.000000   
Developer_back-end                             0.703134  0.578693  0.615301   
Developer_desktop or enterprise applications   0.897516  0.649231  0.194829   
Developer_embedded applications or devices     0.962740  0.625616  0.292627   
Developer_front-end                            0.833829  0.582219  0.364892   
Developer_full-stack                           0.743170  0.676215  0.731455   
Developer_game or graphics                     0.980713  0.375000  0.272727   
Developer_mobile    

In [10]:
classification_report_clustered_features.loc['Mean',:] = classification_report_clustered_features.mean()
classification_report_clustered_features

train                      \
                                              precision    recall  f1_score   
Academic researcher                            0.406650  0.117865  0.182759   
Blockchain                                     0.214286  0.038113  0.064715   
Cloud infrastructure engineer                  0.371233  0.130602  0.193226   
Data or business analyst                       0.363184  0.058776  0.101178   
Data scientist or machine learning specialist  0.633463  0.458592  0.532026   
Database administrator                         0.363636  0.003527  0.006987   
DevOps specialist                              0.454301  0.067681  0.117811   
Developer_QA or test                           0.000000  0.000000  0.000000   
Developer_back-end                             0.576492  0.578074  0.577282   
Developer_desktop or enterprise applications   0.605667  0.119053  0.198991   
Developer_embedded applications or devices     0.542986  0.147783  0.232333   
Developer_front-end                            0.598651  0.323974  0.420425   
Developer_full-stack                           0.668030  0.701748  0.684474   
Developer_game or graphics                     0.425882  0.243607  0.309932   
Developer_mobile                               0.748283  0.519787  0.613448   
Engineer_data                                  0.522105  0.155388  0.239498   
Engineer_site reliability                      0.245455  0.042254  0.072096   
Scientist                                      0.346386  0.143035  0.202465   
Security professional                          0.333333  0.003361  0.006656   
Senior Executive (C-Suite_VP_etc.)             0.000000  0.000000  0.000000   
System administrator                           0.209302  0.006320  0.012270   
full_stack_.NET                                0.708935  0.581161  0.638720   
full_stack_Java                                0.614825  0.478208  0.537978   
full_stack_.JavaScript                         0.685055  0.671839  0.678382   
full_stack_PHP                                 0.668388  0.553891  0.605777   
full_stack_python                              0.601773  0.353564  0.445425   
back_end_Java                                  0.608018  0.555786  0.580730   
back_end_.JavaScript                           0.401054  0.350845  0.374273   
back_end_.NET                                  0.506080  0.474874  0.489981   
back_end_C++                                   0.491152  0.258115  0.338394   
back_end_python                                0.523785  0.296450  0.378613   
back_end_PHP                                   0.508536  0.360133  0.421658   
Mean                                           0.467091  0.274825  0.320578   

                                                             test            \
                                               accuracy precision    recall   
Academic researcher                            0.964674  0.434783  0.116618   
Blockchain                                     0.984921  0.214286  0.042254   
Cloud infrastructure engineer                  0.943782  0.413265  0.148624   
Data or business analyst                       0.967780  0.440000  0.080000   
Data scientist or machine learning specialist  0.964426  0.644654  0.471264   
Database administrator                         0.971754  0.000000  0.000000   
DevOps specialist                              0.937124  0.494737  0.072868   
Developer_QA or test                           0.974065  0.000000  0.000000   
Developer_back-end                             0.685770  0.566702  0.570781   
Developer_desktop or enterprise applications   0.897402  0.675105  0.141218   
Developer_embedded applications or devices     0.960600  0.581197  0.151448   
Developer_front-end                            0.823769  0.600178  0.346351   
Developer_full-stack                           0.730611  0.659775  0.704584   
Developer_game or graphics                     0.979977  0.405405  0.222772   
Developer_mobile    


## 1. From the above results we will choose original features as our features
## 2. we need to also Exclude roles that have f1-score less than 0.1

In [11]:
drop_roles = classification_report_original_features[classification_report_original_features['test']['recall'] < .1].index
drop_roles

Index(['Database administrator', 'Developer_QA or test',
       'Security professional', 'Senior Executive (C-Suite_VP_etc.)',
       'System administrator'],
      dtype='object')

In [12]:
classification_report_original_features = classification_report_original_features.drop(drop_roles)
classification_report_original_features.loc['Mean',:] = classification_report_original_features.mean()
classification_report_original_features

train                      \
                                              precision    recall  f1_score   
Academic researcher                            0.462963  0.220426  0.298656   
Blockchain                                     0.440729  0.272556  0.336818   
Cloud infrastructure engineer                  0.431099  0.217780  0.289375   
Data or business analyst                       0.405914  0.124485  0.190536   
Data scientist or machine learning specialist  0.638930  0.485601  0.551813   
DevOps specialist                              0.504124  0.193357  0.279508   
Developer_back-end                             0.596681  0.621157  0.608673   
Developer_desktop or enterprise applications   0.591538  0.166781  0.260200   
Developer_embedded applications or devices     0.592643  0.265406  0.366625   
Developer_front-end                            0.617082  0.387913  0.476369   
Developer_full-stack                           0.674350  0.731881  0.701939   
Developer_game or graphics                     0.471136  0.338688  0.394081   
Developer_mobile                               0.740981  0.570714  0.644797   
Engineer_data                                  0.519169  0.202998  0.291872   
Engineer_site reliability                      0.355212  0.141757  0.202643   
Scientist                                      0.416667  0.262195  0.321856   
full_stack_.NET                                0.696774  0.624277  0.658537   
full_stack_Java                                0.632667  0.556927  0.592386   
full_stack_.JavaScript                         0.686091  0.704440  0.695144   
full_stack_PHP                                 0.687873  0.643125  0.664747   
full_stack_python                              0.642676  0.548254  0.591722   
back_end_Java                                  0.606329  0.617169  0.611701   
back_end_.JavaScript                           0.457177  0.467685  0.462371   
back_end_.NET                                  0.524378  0.559851  0.541534   
back_end_C++                                   0.548303  0.438321  0.487182   
back_end_python                                0.560851  0.453049  0.501219   
back_end_PHP                                   0.534188  0.492350  0.512416   
Mean                                           0.554993  0.416655  0.461897   

                                                             test            \
                                               accuracy precision    recall   
Academic researcher                            0.965070  0.398810  0.202417   
Blockchain                                     0.985845  0.363636  0.223602   
Cloud infrastructure engineer                  0.943626  0.353571  0.200405   
Data or business analyst                       0.968194  0.370370  0.131579   
Data scientist or machine learning specialist  0.965368  0.656863  0.457859   
DevOps specialist                              0.937503  0.472103  0.179445   
Developer_back-end                             0.703134  0.578693  0.615301   
Developer_desktop or enterprise applications   0.897516  0.649231  0.194829   
Developer_embedded applications or devices     0.962740  0.625616  0.292627   
Developer_front-end                            0.833829  0.582219  0.364892   
Developer_full-stack                           0.743170  0.676215  0.731455   
Developer_game or graphics                     0.980713  0.375000  0.272727   
Developer_mobile                               0.943899  0.736626  0.585605   
Engineer_data                                  0.960905  0.572519  0.196850   
Engineer_site reliability                      0.982052  0.243243  0.107143   
Scientist                                      0.977540  0.257353  0.192308   
full_stack_.NET                                0.900045  0.703312  0.628463   
full_stack_Java                                0.876841  0.642254  0.544153   
full_stack_.JavaScript                         0.855099  0.691596  0.700041   
full_stack_PHP      

In [13]:
classification_report_clustered_features = classification_report_clustered_features.drop(drop_roles)
classification_report_clustered_features.loc['Mean',:] = classification_report_clustered_features.mean()
classification_report_clustered_features

train                      \
                                              precision    recall  f1_score   
Academic researcher                            0.406650  0.117865  0.182759   
Blockchain                                     0.214286  0.038113  0.064715   
Cloud infrastructure engineer                  0.371233  0.130602  0.193226   
Data or business analyst                       0.363184  0.058776  0.101178   
Data scientist or machine learning specialist  0.633463  0.458592  0.532026   
DevOps specialist                              0.454301  0.067681  0.117811   
Developer_back-end                             0.576492  0.578074  0.577282   
Developer_desktop or enterprise applications   0.605667  0.119053  0.198991   
Developer_embedded applications or devices     0.542986  0.147783  0.232333   
Developer_front-end                            0.598651  0.323974  0.420425   
Developer_full-stack                           0.668030  0.701748  0.684474   
Developer_game or graphics                     0.425882  0.243607  0.309932   
Developer_mobile                               0.748283  0.519787  0.613448   
Engineer_data                                  0.522105  0.155388  0.239498   
Engineer_site reliability                      0.245455  0.042254  0.072096   
Scientist                                      0.346386  0.143035  0.202465   
full_stack_.NET                                0.708935  0.581161  0.638720   
full_stack_Java                                0.614825  0.478208  0.537978   
full_stack_.JavaScript                         0.685055  0.671839  0.678382   
full_stack_PHP                                 0.668388  0.553891  0.605777   
full_stack_python                              0.601773  0.353564  0.445425   
back_end_Java                                  0.608018  0.555786  0.580730   
back_end_.JavaScript                           0.401054  0.350845  0.374273   
back_end_.NET                                  0.506080  0.474874  0.489981   
back_end_C++                                   0.491152  0.258115  0.338394   
back_end_python                                0.523785  0.296450  0.378613   
back_end_PHP                                   0.508536  0.360133  0.421658   
Mean                                           0.518134  0.323429  0.376899   

                                                             test            \
                                               accuracy precision    recall   
Academic researcher                            0.964674  0.434783  0.116618   
Blockchain                                     0.984921  0.214286  0.042254   
Cloud infrastructure engineer                  0.943782  0.413265  0.148624   
Data or business analyst                       0.967780  0.440000  0.080000   
Data scientist or machine learning specialist  0.964426  0.644654  0.471264   
DevOps specialist                              0.937124  0.494737  0.072868   
Developer_back-end                             0.685770  0.566702  0.570781   
Developer_desktop or enterprise applications   0.897402  0.675105  0.141218   
Developer_embedded applications or devices     0.960600  0.581197  0.151448   
Developer_front-end                            0.823769  0.600178  0.346351   
Developer_full-stack                           0.730611  0.659775  0.704584   
Developer_game or graphics                     0.979977  0.405405  0.222772   
Developer_mobile                               0.942018  0.761981  0.500525   
Engineer_data                                  0.960873  0.382114  0.121762   
Engineer_site reliability                      0.982735  0.242424  0.044944   
Scientist                                      0.977493  0.395062  0.161616   
full_stack_.NET                                0.897874  0.697744  0.593730   
full_stack_Java                                0.866572  0.608187  0.449660   
full_stack_.JavaScript                         0.849058  0.667092  0.659521   
full_stack_PHP      

In [14]:
job_names = classification_report_clustered_features[:-1].index
job_names = list(job_names)
job_names

['Academic researcher',
 'Blockchain',
 'Cloud infrastructure engineer',
 'Data or business analyst',
 'Data scientist or machine learning specialist',
 'DevOps specialist',
 'Developer_back-end',
 'Developer_desktop or enterprise applications',
 'Developer_embedded applications or devices',
 'Developer_front-end',
 'Developer_full-stack',
 'Developer_game or graphics',
 'Developer_mobile',
 'Engineer_data',
 'Engineer_site reliability',
 'Scientist',
 'full_stack_.NET',
 'full_stack_Java',
 'full_stack_.JavaScript',
 'full_stack_PHP',
 'full_stack_python',
 'back_end_Java',
 'back_end_.JavaScript',
 'back_end_.NET',
 'back_end_C++',
 'back_end_python',
 'back_end_PHP']

# 6. Concat skills_df with DevType

In [15]:
skills_df = skills_dev_df.drop('clustered_skills', axis = 1, level=0).drop('DevType', axis = 1, level = 0)

In [16]:
dev_df = skills_dev_df['DevType'].drop(drop_roles, axis = 1)
dev_df.columns = pd.MultiIndex.from_product([['DevType'],job_names])


In [17]:
skills_dev_df = pd.concat([skills_df, dev_df], axis=1, join='inner')
skills_dev_df

Languages                                                             \
            APL Assembly Bash/Shell    C    C#   C++ COBOL Clojure Crystal   
2           0.0      0.0        0.0  0.0  0.75  0.75   0.0     0.0     0.0   
3           0.0      0.0        0.0  0.0  1.50  0.00   0.0     0.0     0.0   
7           0.5      0.0        0.5  1.0  0.00  1.00   0.0     0.0     0.0   
9           0.0      0.0        0.0  0.0  0.00  0.00   0.0     0.0     0.0   
10          0.0      0.0        1.0  0.0  1.50  0.00   0.0     0.0     0.0   
...         ...      ...        ...  ...   ...   ...   ...     ...     ...   
73262       0.0      0.5        0.0  0.5  0.00  0.75   0.0     0.0     0.0   
73263       0.0      0.0        1.5  0.0  0.00  0.00   0.0     0.0     0.0   
73264       0.0      0.0        1.0  0.0  0.00  0.00   0.0     0.0     0.0   
73265       0.0      0.0        0.0  0.0  0.25  0.00   0.0     0.0     0.0   
73266       0.0      0.0        0.0  0.0  1.00  0.00   0.0     0.0     0.0   

            ...         DevType                                        \
      Dart  ... full_stack_Java full_stack_.JavaScript full_stack_PHP   
2      0.0  ...               0                      0              0   
3      0.0  ...               0                      0              0   
7      0.0  ...               1                      1              1   
9      0.0  ...               0                      0              0   
10     0.0  ...               0                      0              0   
...    ...  ...             ...                    ...            ...   
73262  0.0  ...               0                      0              0   
73263  1.0  ...               0                      0              0   
73264  0.0  ...               0                      0              0   
73265  0.0  ...               0                      0              1   
73266  0.0  ...               0                      0              0   

                                                                          \
      full_stack_python back_end_Java back_end_.JavaScript back_end_.NET   
2                     0             0                    0             0   
3                     0             0                    0             0   
7                     0             0                    0             0   
9                     0             0                    0             0   
10                    0             0                    0             1   
...                 ...           ...                  ...           ...   
73262                 0             0                    0             0   
73263                 0             0                    1             0   
73264                 0             0                    0             0   
73265                 1             0                    0             0   
73266                 0             0                    0             0   

                                                 
      back_end_C++ back_end_python back_end_PHP  
2                0               0            0  
3                0               0            0  
7                0               0            0  
9                0               0            0  
10               0               0            0  
...            ...             ...          ...  
73262            0               0            0  
73263            1               1            1  
73264            0               0            0  
73265            0               0            0  
73266            0               0            0  

[50423 rows x 188 columns]

In [18]:
skills_dev_df.to_pickle(LOAD_PATH + SAVE_DF_NAME)


# Save logs

## 1. original features

In [19]:
classification_report_original_features

train                      \
                                              precision    recall  f1_score   
Academic researcher                            0.462963  0.220426  0.298656   
Blockchain                                     0.440729  0.272556  0.336818   
Cloud infrastructure engineer                  0.431099  0.217780  0.289375   
Data or business analyst                       0.405914  0.124485  0.190536   
Data scientist or machine learning specialist  0.638930  0.485601  0.551813   
DevOps specialist                              0.504124  0.193357  0.279508   
Developer_back-end                             0.596681  0.621157  0.608673   
Developer_desktop or enterprise applications   0.591538  0.166781  0.260200   
Developer_embedded applications or devices     0.592643  0.265406  0.366625   
Developer_front-end                            0.617082  0.387913  0.476369   
Developer_full-stack                           0.674350  0.731881  0.701939   
Developer_game or graphics                     0.471136  0.338688  0.394081   
Developer_mobile                               0.740981  0.570714  0.644797   
Engineer_data                                  0.519169  0.202998  0.291872   
Engineer_site reliability                      0.355212  0.141757  0.202643   
Scientist                                      0.416667  0.262195  0.321856   
full_stack_.NET                                0.696774  0.624277  0.658537   
full_stack_Java                                0.632667  0.556927  0.592386   
full_stack_.JavaScript                         0.686091  0.704440  0.695144   
full_stack_PHP                                 0.687873  0.643125  0.664747   
full_stack_python                              0.642676  0.548254  0.591722   
back_end_Java                                  0.606329  0.617169  0.611701   
back_end_.JavaScript                           0.457177  0.467685  0.462371   
back_end_.NET                                  0.524378  0.559851  0.541534   
back_end_C++                                   0.548303  0.438321  0.487182   
back_end_python                                0.560851  0.453049  0.501219   
back_end_PHP                                   0.534188  0.492350  0.512416   
Mean                                           0.554993  0.416655  0.461897   

                                                             test            \
                                               accuracy precision    recall   
Academic researcher                            0.965070  0.398810  0.202417   
Blockchain                                     0.985845  0.363636  0.223602   
Cloud infrastructure engineer                  0.943626  0.353571  0.200405   
Data or business analyst                       0.968194  0.370370  0.131579   
Data scientist or machine learning specialist  0.965368  0.656863  0.457859   
DevOps specialist                              0.937503  0.472103  0.179445   
Developer_back-end                             0.703134  0.578693  0.615301   
Developer_desktop or enterprise applications   0.897516  0.649231  0.194829   
Developer_embedded applications or devices     0.962740  0.625616  0.292627   
Developer_front-end                            0.833829  0.582219  0.364892   
Developer_full-stack                           0.743170  0.676215  0.731455   
Developer_game or graphics                     0.980713  0.375000  0.272727   
Developer_mobile                               0.943899  0.736626  0.585605   
Engineer_data                                  0.960905  0.572519  0.196850   
Engineer_site reliability                      0.982052  0.243243  0.107143   
Scientist                                      0.977540  0.257353  0.192308   
full_stack_.NET                                0.900045  0.703312  0.628463   
full_stack_Java                                0.876841  0.642254  0.544153   
full_stack_.JavaScript                         0.855099  0.691596  0.700041   
full_stack_PHP      

In [20]:
original_features = original_df.drop('DevType',level=0, axis=1).droplevel(level=0,axis=1).columns
original_modeling_utils.save_results(all_classification_report=classification_report_original_features,
                                     model_name='Logistic-Regression_Original-Features_1.0.0',
                                     data_path=LOAD_SKILLS_DEV, features=original_features)


clustered_features = clustered_df.drop('DevType',level=0, axis=1).droplevel(level=0,axis=1).columns
clustered_modeling_utils.save_results(all_classification_report=classification_report_clustered_features,
                                     model_name='Logistic-Regression_Clustered-Features_1.0.0',
                                     data_path=LOAD_SKILLS_DEV, features=clustered_features)

2023/08/17 12:25:36 INFO mlflow.tracking.fluent: Experiment with name 'skills_rec_analysis' does not exist. Creating a new experiment.


In [21]:
MlflowUtils.get_runs()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,metrics.recall,metrics.f1_score,metrics.precision,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.runName,tags.mlflow.source.name
0,b4928cc3e3bf4f21b8b9138310447d2c,735982700926372623,FINISHED,file:///D:/protofolio_projects/Dev_skills_reco...,2023-08-17 09:25:36.899000+00:00,2023-08-17 09:25:36.934000+00:00,0.894298,0.324855,0.378904,0.522859,mahmo,LOCAL,Logistic-Regression_Clustered-Features_1.0.0,C:\Users\mahmo\anaconda3\envs\Rec-skills\lib\s...
1,91321ce64e8646d7a8e28dac0a5d8deb,735982700926372623,FINISHED,file:///D:/protofolio_projects/Dev_skills_reco...,2023-08-17 09:25:36.843000+00:00,2023-08-17 09:25:36.877000+00:00,0.897505,0.404598,0.446315,0.531665,mahmo,LOCAL,Logistic-Regression_Original-Features_1.0.0,C:\Users\mahmo\anaconda3\envs\Rec-skills\lib\s...


### as we see from the above dataframe, original features outperform clustered features in all metricises especially recall & f1_score that we care more
#### 1. so will use the original features not all the clustered features
#### 2. And we will use These metricises as reference for any further complex model